In [64]:
import openai
from dotenv import load_dotenv
import os

dotenv_path = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path)

openai_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_key)

In [1]:
import yfinance as yf

ticker = yf.Ticker("AAPL")
hist = ticker.history(period="1mo")
print(hist)

                                 Open        High         Low       Close  \
Date                                                                        
2024-10-01 00:00:00-04:00  229.520004  229.649994  223.740005  226.210007   
2024-10-02 00:00:00-04:00  225.889999  227.369995  223.020004  226.779999   
2024-10-03 00:00:00-04:00  225.139999  226.809998  223.320007  225.669998   
2024-10-04 00:00:00-04:00  227.899994  228.000000  224.130005  226.800003   
2024-10-07 00:00:00-04:00  224.500000  225.690002  221.330002  221.690002   
2024-10-08 00:00:00-04:00  224.300003  225.979996  223.250000  225.770004   
2024-10-09 00:00:00-04:00  225.229996  229.750000  224.830002  229.539993   
2024-10-10 00:00:00-04:00  227.779999  229.500000  227.169998  229.039993   
2024-10-11 00:00:00-04:00  229.300003  229.410004  227.339996  227.550003   
2024-10-14 00:00:00-04:00  228.699997  231.729996  228.600006  231.300003   
2024-10-15 00:00:00-04:00  233.610001  237.490005  232.369995  233.850006   

In [20]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import re

def custom_round(x):
    if isinstance(x, (int, float)):
        rounded = round(x, 1)
        return int(rounded) if rounded.is_integer() else rounded
    return x

def get_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)

    end_date_extended = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=50)).strftime("%Y-%m-%d")
    df = stock.history(start=start_date, end=end_date_extended, auto_adjust=False)
    
    # 데이터프레임 리셋 및 날짜 컬럼 추가
    df = df.reset_index()
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    
    # 필요한 컬럼만 선택
    df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close']]
    
    # inc-5, inc-10 등의 컬럼 추가
    for days in [5, 10, 15, 20, 25, 30]:
        df[f'inc-{days}'] = df['Adj Close'].pct_change(periods=days).shift(-days) * 100

    result = df[:10].applymap(custom_round)
    
    return result

In [58]:
def df_to_str(df):
    # 인덱스를 열로 변환
    df = df.reset_index()
    df.columns = df.columns.str.lower()
    df = df.rename(columns={'index': ''})
    
    # col_widths = [max(len(str(col)), df[col].astype(str).map(len).max()) for col in df.columns]
    col_widths = [2, 10, 6, 6, 5, 7, 11, 7, 8, 8, 8, 8, 8]
    
    # 헤더 생성
    header_list = []
    for col, width in zip(df.columns, col_widths):
        if col == 'date':
            header_text = f" {col:<{width}} "    
        else:
            header_text = f" {col:>{width}} "
        header_list.append(header_text)
        
    header = "|" + "|".join(header_list) + "|"
    
    # 구분선 생성
    seperator_list = []
    for col, width in zip(df.columns, col_widths):
        seperator_text = "-" * (width + 1)

        if col == 'date':
            seperator_text = ':' + seperator_text 
        else:
            seperator_text += ':' 

        seperator_list.append(seperator_text)

    separator = "|" + "|".join(seperator_list) + "|"
    
    # 데이터 행 생성
    data_rows = []
    for _, row in df.iterrows():
        data_row = "|" + "|".join(f" {str(val):>{width}} " for val, width in zip(row, col_widths)) + "|"
        data_rows.append(data_row)
    
    # 최종 결과 조합
    final_result = "\n".join([header, separator] + data_rows)
    
    return final_result

In [59]:
import random
from datetime import datetime, timedelta

# 날짜 랜덤 뽑기
def random_date_range_2023():
    year_start = datetime(2023, 1, 1)
    year_end = datetime(2024, 6, 30)
    
    max_start = year_end - timedelta(days=11)
    random_days = random.randint(0, (max_start - year_start).days)
    start_date = year_start + timedelta(days=random_days)
    
    end_date = start_date + timedelta(days=11)
    
    start_str = start_date.strftime("%Y-%m-%d")
    end_str = end_date.strftime("%Y-%m-%d")
    
    return start_str, end_str

In [69]:
def get_temp_answer(df):
    score = sum(df.iloc[-1][["inc-5", "inc-10", "inc-15", "inc-20", "inc-25", "inc-30"]])

    if score > 2:
        return '증가'
    elif score < -2:
        return '감소'
    else:
        return None

In [65]:
def get_answer(question):
    system_prompt = "다음 표는 주식 A에 대한 분석 결과입니다. 표를 바탕으로 향후 A의 주가가 상승할지 하락할지 예측하고 [증가] 혹은 [감소] 라고 답변하시요."
    system_msg = {"role":"system", "content":system_prompt}

    user_msg = {"role":'user', 'content': question}

    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = [system_msg, user_msg]
    )

    result = response.choices[0].message.content

    print(result)
    if '감소' in result:
        return '[감소]'
    elif '증가' in result:
        return '[증가]'
    else:
        return None

In [70]:
ticker = "AAPL"  # Apple Inc.
start_date, end_date = random_date_range_2023()

df = get_stock_data(ticker, start_date, end_date)
temp_answer = get_temp_answer(df)
result = "### 분석:\n" + df_to_str(df)
print(result)
print(temp_answer)

### 분석:
|    | date       |   open |   high |   low |   close |   adj close |   inc-5 |   inc-10 |   inc-15 |   inc-20 |   inc-25 |   inc-30 |
|---:|:-----------|-------:|-------:|------:|--------:|------------:|--------:|---------:|---------:|---------:|---------:|---------:|
|  0 | 2024-02-12 |  188.4 |  188.7 | 186.8 |   187.1 |       186.7 |    -3.0 |     -2.4 |     -9.1 |     -7.4 |     -5.9 |     -9.3 |
|  1 | 2024-02-13 |  185.8 |  186.2 | 183.5 |   185.0 |       184.6 |    -1.5 |     -2.0 |     -8.6 |     -7.5 |     -3.4 |     -6.3 |
|  2 | 2024-02-14 |  185.3 |  185.5 | 182.4 |   184.1 |       183.7 |     0.1 |     -1.8 |     -8.2 |     -6.1 |     -6.9 |     -6.9 |
|  3 | 2024-02-15 |  183.6 |  184.5 | 181.4 |   183.9 |       183.4 |    -0.7 |     -2.3 |     -7.1 |     -6.1 |     -6.3 |     -7.5 |
|  4 | 2024-02-16 |  183.4 |  184.9 | 181.7 |   182.3 |       181.9 |    -0.6 |     -4.0 |     -5.2 |     -4.7 |     -6.3 |     -7.4 |
|  5 | 2024-02-20 |  181.8 |  182.4 | 180.0 |  

/tmp/ipykernel_96328/2816013016.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result = df[:10].applymap(custom_round)


# 데이터를 뽑자

In [56]:
korea_tickers = [
    '005930.KS',  # 삼성전자
    '000660.KS',  # SK하이닉스
    '005380.KS',  # 현대차
    '035420.KS',  # NAVER
    '051910.KS',  # LG화학
    '005490.KS',  # POSCO홀딩스
    '035720.KS',  # 카카오
    '000270.KS',  # 기아
    '006400.KS',  # 삼성SDI
    '068270.KS',  # 셀트리온
    '051900.KS',  # LG생활건강
    '028260.KS',  # 삼성물산
    '207940.KS',  # 삼성바이오로직스
    '012330.KS',  # 현대모비스
    '066570.KS',  # LG전자
    '032830.KS',  # 삼성생명
    '055550.KS',  # 신한지주
    '015760.KS',  # 한국전력
    '017670.KS',  # SK텔레콤
    '034730.KS',  # SK
    '003550.KS',  # LG
    '105560.KS',  # KB금융
    '086790.KS',  # 하나금융지주
    '033780.KS',  # KT&G
    '000810.KS',  # 삼성화재
    '009150.KS',  # 삼성전기
    '018260.KS',  # 삼성에스디에스
    '036570.KS',  # 엔씨소프트
    '096770.KS',  # SK이노베이션
    '090430.KS',  # 아모레퍼시픽
    '316140.KS',  # 우리금융지주
    '011200.KS',  # HMM
    '024110.KS',  # 기업은행
    '010130.KS',  # 고려아연
    '009540.KS',  # 한국조선해양
    '010950.KS',  # S-Oil
    '034220.KS',  # LG디스플레이
    '086280.KS',  # 현대글로비스
    '000720.KS',  # 현대건설
    '047810.KS',  # 한국항공우주
    '161390.KS',  # 한국타이어앤테크놀로지
    '023530.KS',  # 롯데쇼핑
    '032640.KS',  # LG유플러스
    '011170.KS',  # 롯데케미칼
    '035250.KS',  # 강원랜드
    '021240.KS',  # 코웨이
    '004020.KS',  # 현대제철
    '139480.KS',  # 이마트
    '097950.KS',  # CJ제일제당
    '006800.KS',  # 미래에셋증권
    '035760.KQ',  # CJ ENM
    '251270.KS',  # 넷마블
    '006280.KS',  # 녹십자
    '011780.KS',  # 금호석유
    '003490.KS',  # 대한항공
    '008770.KS',  # 호텔신라
    '005830.KS',  # DB손해보험
    '128940.KS',  # 한미약품
    '138040.KS',  # 메리츠금융지주
    '011790.KS',  # SKC
    '005940.KS',  # NH투자증권
    '078930.KS',  # GS
    '006360.KS',  # GS건설
    '009830.KS',  # 한화솔루션
    '000100.KS',  # 유한양행
    '030200.KS',  # KT
    '002790.KS',  # 아모레G
    '004990.KS',  # 롯데지주
    '001040.KS',  # CJ
    '010060.KS',  # OCI
    '000120.KS',  # CJ대한통운
    '069960.KS',  # 현대백화점
    '004170.KS',  # 신세계
    '010140.KS',  # 삼성중공업
    '001450.KS',  # 현대해상
    '011070.KS',  # LG이노텍
    '036460.KS',  # 한국가스공사
    '012750.KS',  # S&T모티브
    '282330.KS',  # BGF리테일
    '008930.KS',  # 한미사이언스
    '009240.KS',  # 한샘
    '007070.KS',  # GS리테일
    '002380.KS',  # KCC
    '039490.KS',  # 키움증권
    '001740.KS',  # SK네트웍스
    '005300.KS',  # 롯데칠성
    '000080.KS',  # 하이트진로
    '004800.KS',  # 효성
    '001800.KS',  # 오리온홀딩스
    '003410.KS',  # 쌍용C&E
    '004370.KS',  # 농심
    '285130.KS',  # SK케미칼
    '003690.KS',  # 코오롱인더
    '000210.KS',  # DL
    '002710.KS',  # TCC스틸
    '005850.KS',  # 에스엘
    '006650.KS',  # 대한유화
    '001680.KS',  # 대상
    '003000.KS',  # 부광약품
    '192820.KS',  # 코스맥스
]

In [66]:
# 8.10 -> 8.09 
from tqdm import tqdm

stock_price_questions = []
for ticker in tqdm(korea_tickers):
    decrease_data_cnt = 0 
    increase_data_cnt = 0

    for _ in range(10):
        start_date, end_date = random_date_range_2023()
        df = get_stock_data(ticker, start_date, end_date)
        temp_answer = get_temp_answer(df)

        if temp_answer == '증가':

        elif temp_answer == '감소':
            

    question = "### 분석:\n" + df_to_str(df)
    answer = get_answer(question)
    item = {"question": question, "answer": answer}
    stock_price_questions.append(item)        

/tmp/ipykernel_96328/2816013016.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result = df[:10].applymap(custom_round)


향후 A의 주가는 [감소]할 것으로 예측됩니다.


/tmp/ipykernel_96328/2816013016.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result = df[:10].applymap(custom_round)


[감소]


/tmp/ipykernel_96328/2816013016.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result = df[:10].applymap(custom_round)


...

[증가]


/tmp/ipykernel_96328/2816013016.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result = df[:10].applymap(custom_round)


주식 A의 최근 분석 결과를 살펴보면, 10일, 15일, 20일, 25일 기간 모두에서 주가가 감소하는 추세를 보이고 있습니다. 특히, 'inc-5'에서 '-10.4', 'inc-10'에서 '-3.0', 'inc-15'에서 '2.3' 등의 수치가 나타나며 전반적으로 부정적인 흐름이 지속되고 있습니다.

따라서 향후 주가가 어떻게 될지를 예측하면 주가는 [감소]할 것으로 보입니다.


/tmp/ipykernel_96328/2816013016.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result = df[:10].applymap(custom_round)


[감소]
